# Rozpoznawanie nazwanych jednostek (NER)

Ten notatnik pochodzi z [AI for Beginners Curriculum](http://aka.ms/ai-beginners).

W tym przykładzie nauczymy się, jak trenować model NER na zbiorze danych [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) z Kaggle. Przed rozpoczęciem proszę pobrać plik [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) do bieżącego katalogu.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Przygotowanie zestawu danych

Zaczniemy od wczytania zestawu danych do ramki danych. Jeśli chcesz dowiedzieć się więcej o korzystaniu z Pandas, odwiedź [lekcję o przetwarzaniu danych](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) w naszym [Data Science dla początkujących](http://aka.ms/datascience-beginners)


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Zdobądźmy unikalne tagi i stwórzmy słowniki wyszukiwania, które możemy wykorzystać do konwersji tagów na numery klas:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Teraz musimy zrobić to samo ze słownictwem. Dla uproszczenia stworzymy słownictwo bez uwzględniania częstotliwości występowania słów; w rzeczywistości możesz chcieć użyć wektoryzatora Keras i ograniczyć liczbę słów.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Musimy stworzyć zbiór danych z zdaniami do treningu. Przejdźmy przez oryginalny zbiór danych i rozdzielmy wszystkie pojedyncze zdania na `X` (listy słów) i `Y` (listy tokenów):


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Dla uproszczenia, będziemy uzupełniać wszystkie zdania 0 tokenami do maksymalnej długości. W rzeczywistości moglibyśmy zastosować bardziej sprytną strategię i uzupełniać sekwencje tylko w obrębie jednej minibatch.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Definiowanie sieci klasyfikacji tokenów

Użyjemy dwuwarstwowej sieci LSTM o kierunkach dwukierunkowych do klasyfikacji tokenów. Aby zastosować gęsty klasyfikator do każdego z wyjść ostatniej warstwy LSTM, użyjemy konstrukcji `TimeDistributed`, która replikuje tę samą gęstą warstwę dla każdego z wyjść LSTM na każdym kroku:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

W tym miejscu określamy `maxlen` dla naszego zbioru danych - jeśli chcemy, aby sieć mogła obsługiwać sekwencje o zmiennej długości, musimy być nieco bardziej sprytni przy definiowaniu sieci.

Teraz przejdźmy do trenowania modelu. Dla szybkości przeprowadzimy trening tylko przez jedną epokę, ale możesz spróbować trenować przez dłuższy czas. Ponadto warto rozdzielić część zbioru danych jako zbiór treningowy, aby obserwować dokładność walidacji.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Testowanie Wyniku

Sprawdźmy teraz, jak nasz model rozpoznawania jednostek działa na przykładowym zdaniu:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Kluczowe informacje

Nawet prosty model LSTM daje przyzwoite wyniki w rozpoznawaniu nazwanych jednostek (NER). Jednak aby uzyskać znacznie lepsze rezultaty, warto skorzystać z dużych, wstępnie wytrenowanych modeli językowych, takich jak BERT. Proces trenowania BERT dla NER przy użyciu biblioteki Huggingface Transformers został opisany [tutaj](https://huggingface.co/course/chapter7/2?fw=pt).



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
